<a href="https://colab.research.google.com/github/iliakhosravi/pneumonia_detection/blob/main/ilia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()

Saving FinalDataset.zip to FinalDataset.zip


In [2]:
!unzip FinalDataset.zip

Archive:  FinalDataset.zip
   creating: FinalDataset/
   creating: FinalDataset/normal/
  inflating: FinalDataset/normal/1 (1).jfif  
  inflating: FinalDataset/normal/1 (10).jfif  
  inflating: FinalDataset/normal/1 (11).jfif  
  inflating: FinalDataset/normal/1 (12).jfif  
  inflating: FinalDataset/normal/1 (13).jfif  
  inflating: FinalDataset/normal/1 (14).jfif  
  inflating: FinalDataset/normal/1 (15).jfif  
  inflating: FinalDataset/normal/1 (16).jfif  
  inflating: FinalDataset/normal/1 (17).jfif  
  inflating: FinalDataset/normal/1 (18).jfif  
  inflating: FinalDataset/normal/1 (19).jfif  
  inflating: FinalDataset/normal/1 (2).jfif  
  inflating: FinalDataset/normal/1 (20).jfif  
  inflating: FinalDataset/normal/1 (21).jfif  
  inflating: FinalDataset/normal/1 (22).jfif  
  inflating: FinalDataset/normal/1 (23).jfif  
  inflating: FinalDataset/normal/1 (24).jfif  
  inflating: FinalDataset/normal/1 (25).jfif  
  inflating: FinalDataset/normal/1 (26).jfif  
  inflating: FinalDat

In [8]:
!pwd FinalDataset

/content


In [14]:
!mkdir FinalDatasetConverted/normal
!mkdir FinalDatasetConverted/pneumonia

mkdir: cannot create directory ‘FinalDatasetConverted/normal’: File exists
mkdir: cannot create directory ‘FinalDatasetConverted/pneumonia’: File exists


In [15]:
!ls FinalDatasetConverted

normal	pneumonia


In [18]:
# Importing Library
from PIL import Image
import os
input_dir_normal = "/content/FinalDataset/normal"
input_dir_pneumonia = "/content/FinalDataset/pneumonia"
output_dir_normal = "/content/FinalDatasetConverted/normal"
output_dir_pneumonia = "/content/FinalDatasetConverted/pneumonia"
# Loading the image
for filename in os.listdir(input_dir_normal):
  if filename.endswith(".jfif"):
    image = Image.open(os.path.join(input_dir_normal, filename))
    output_filename = os.path.splitext(filename)[0] + ".jpg"
    # Converting image from JFIF to JPG format
  image.save(os.path.join(output_dir_normal, output_filename), "JPEG")
print("Normal Images successfully converted!")
for filename in os.listdir(input_dir_pneumonia):
  if filename.endswith(".jfif"):
    image = Image.open(os.path.join(input_dir_pneumonia, filename))
    output_filename = os.path.splitext(filename)[0] + ".jpg"
    # Converting image from JFIF to JPG format
  image.save(os.path.join(output_dir_pneumonia, output_filename), "JPEG")
print("Pneumonia Images successfully converted!")


Normal Images successfully converted!
Pneumonia Images successfully converted!


In [19]:
!du -h FinalDatasetConverted

20M	FinalDatasetConverted/normal
90M	FinalDatasetConverted/pneumonia
110M	FinalDatasetConverted


In [21]:
!pip install tensorflow-addons
import tensorflow_addons as tfa
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# from spp.SpatialPyramidPooling import SpatialPyramidPooling # Ilia

from tensorflow.keras.metrics import FalsePositives, FalseNegatives, TrueNegatives, TruePositives

batch_size = 32
img_height = 100
img_width = 143

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'FinalDatasetConverted',
  validation_split=0.1,
  subset="training",
  seed=123,
  # image_size=(img_height, img_width),
  batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'FinalDatasetConverted',
  validation_split=0.1,
  subset="validation",
  seed=123,
  # image_size=(img_height, img_width),
  batch_size=batch_size
)

model = keras.models.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(2, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(4, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(8, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  tfa.layers.SpatialPyramidPooling2D([1, 2, 4]),
  layers.Flatten(),
  #layers.SpatialPyramidPooling([1, 2, 4]), # Ilia

  layers.Dense(32, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

Found 909 files belonging to 2 classes.
Using 819 files for training.
Found 909 files belonging to 2 classes.
Using 90 files for validation.


In [22]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


#EPOCHS = 6
#earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
#mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
#reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')

#model.compile(optimizer='rmsprop',
#             loss='binary_crossentropy',
#              metrics=['accuracy'])


#history = model.fit(
#        train_ds,
#        validation_data=val_ds,
#        epochs=EPOCHS,
#        verbose=1,
#        callbacks=[earlyStopping, mcp_save, reduce_lr_loss],
#    )

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=3
)

Epoch 1/3
26/26 [==============================] - 47s 1s/step - loss: 0.5200 - accuracy: 0.7106 - val_loss: 0.4052 - val_accuracy: 0.7444
Epoch 2/3
26/26 [==============================] - 34s 1s/step - loss: 0.3186 - accuracy: 0.8462 - val_loss: 0.2440 - val_accuracy: 0.9000
Epoch 3/3
26/26 [==============================] - 37s 1s/step - loss: 0.2292 - accuracy: 0.9145 - val_loss: 0.1952 - val_accuracy: 0.9222
